### Debt Repayment Plan

#### Part 1 - Basic Case

- Data
    - **N**  : Net monthly income
    - **M**  : Monthly required savings
    - **I**  : Interest paid on loan
    

- Decision Variables
    - **S<sub>i</sub>**  : Supplementary monthly savings during period **i**
    - **P<sub>i</sub>**  : Debt paid during month **i**
    - **B<sub>i</sub>**  : Debt owed at the beginning of month **i**

In [1]:
using JuMP
import HiGHS

In [2]:
# Data
data = Dict(
    "debt" => Dict(
        "loan" => 10000,
        "term" => 10,
        "interest" => 6.8 / 100
    ),
    "net_income" => 4500,
    "minimum_savings" => 1000
)
periods = 1:data["debt"]["term"]


1:10

In [3]:
function set_model_variables(model::Model, periods::UnitRange, NET_INCOME::Number, MINIMUM_SAVINGS::Number)
    # Monthly savings
    Savings = @variable(model, S[periods] >= MINIMUM_SAVINGS );

    # Monthly debt reimbursed
    Payments = @variable(model, P[periods] >= 0 );

    # Monthly balance
    Balance = @variable(model, B[cat(periods, periods[end] + 1, dims = 1)] >= 0);
    
    return Savings, Payments, Balance
end

set_model_variables (generic function with 1 method)

In [4]:
function set_model_constraints( 
        model::Model, 
        debt::Dict,
        NET_INCOME::Int,
        Savings::JuMP.Containers.DenseAxisArray, 
        Payments::JuMP.Containers.DenseAxisArray, 
        Balance::JuMP.Containers.DenseAxisArray
    )
    loan_label = "loan"; period_label = "term"; interest_label = "interest";
    if haskey(debt, loan_label ) && haskey( debt, period_label ) && haskey( debt, interest_label )
        # Debt owed at the beginning of period 1
        @constraint( model, Balance[1] == debt[loan_label] )

        # Ensure that all debt is paid off by the grace period
        @constraint( model, Balance[debt[period_label] + 1] == 0 )

        # Debt owed the beginning of period 2...end
        @constraint( model, [i in periods], Balance[i + 1] == (1 + debt[interest_label]) * (Balance[i] - Payments[i]) )

        # Monthly expense can't exceed net income
        @constraint( model, [i in periods], Savings[i] + Payments[i] <= NET_INCOME )
    end
end

set_model_constraints (generic function with 1 method)

#### Solve Model

In [5]:
model = Model( HiGHS.Optimizer )

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

In [6]:
Savings, Payments, Balance = set_model_variables( model, periods, data["net_income"], data["minimum_savings"])

(1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{VariableRef}:
 S[1]
 S[2]
 S[3]
 S[4]
 S[5]
 S[6]
 S[7]
 S[8]
 S[9]
 S[10], 1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{VariableRef}:
 P[1]
 P[2]
 P[3]
 P[4]
 P[5]
 P[6]
 P[7]
 P[8]
 P[9]
 P[10], 1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
And data, a 11-element Vector{VariableRef}:
 B[1]
 B[2]
 B[3]
 B[4]
 B[5]
 B[6]
 B[7]
 B[8]
 B[9]
 B[10]
 B[11])

In [7]:
set_model_constraints( model, data["debt"], data["net_income"], Savings, Payments, Balance )

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 S[1] + P[1] <= 4500.0
 S[2] + P[2] <= 4500.0
 S[3] + P[3] <= 4500.0
 S[4] + P[4] <= 4500.0
 S[5] + P[5] <= 4500.0
 S[6] + P[6] <= 4500.0
 S[7] + P[7] <= 4500.0
 S[8] + P[8] <= 4500.0
 S[9] + P[9] <= 4500.0
 S[10] + P[10] <= 4500.0

In [8]:
@objective(model, Max, sum(Savings))

S[1] + S[2] + S[3] + S[4] + S[5] + S[6] + S[7] + S[8] + S[9] + S[10]

In [9]:
print( model )

In [10]:
function solve_infeasible(model)
    optimize!(model)
    if termination_status(model) == OPTIMAL
        return objective_value(model)
    else
        @warn("The model was not solved correctly.")
        return nothing
    end
end

solve_infeasible(model)

Presolving model
18 rows, 27 cols, 44 nonzeros
1 rows, 10 cols, 10 nonzeros
1 rows, 10 cols, 10 nonzeros
Presolve : Reductions: rows 1(-21); columns 10(-21); elements 10(-42)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.5000000000e+04 Pr: 1(12181.9) 0s
          1    -3.4311972192e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1
Objective value     :  3.4311972192e+04
HiGHS run time      :          0.00


34311.972192

In [18]:
using Printf
function print_payment_plan( 
        model::Model,
        debt::Dict,
        Savings::JuMP.Containers.DenseAxisArray, 
        Payments::JuMP.Containers.DenseAxisArray, 
        Balance::JuMP.Containers.DenseAxisArray
    )
    @printf("| %-10s | %-15s | %-10s | %-10s | %-15s | %-10s | %-12s | %-10s |\n",
        "Period", "Initial Balance", "Payment", "Savings", 
        "Int. Rate (%)", "Interest", "End Balance", "Term"
    )
    cumulative_savings = cumsum([value(Savings[i]) for i in periods])
    for i in periods
        start_balance = value(Balance[i])
        end_balance = value(Balance[i]) - value(Payments[i])
        payment = value(Payments[i])
        interest = ( 
            i == 1 ? 0 
            : 
            value(Balance[i]) - (value(Balance[i - 1] ) - value(Payments[i - 1]))
        )
        @printf("| %10d | %15.2f | %10.2f | %10.2f | %15.2f | %10.2f | %12.2f | %10d |\n", 
            i,
            start_balance > 0 ? start_balance : 0.00,
            payment > 0 ? payment : 0.00, 
            cumulative_savings[i],
            debt["interest"],
            interest > 0 ? interest : 0.00,
            end_balance > 0 ? end_balance : 0.00,
            debt["term"]
        )
    end
end;

In [19]:
if termination_status(model) == OPTIMAL
    print_payment_plan(model, data["debt"], Savings, Payments, Balance)
end

| Period     | Initial Balance | Payment    | Savings    | Int. Rate (%)   | Interest   | End Balance  | Term       |
|          1 |        10000.00 |    3500.00 |    1000.00 |            0.07 |       0.00 |      6500.00 |         10 |
|          2 |         6942.00 |    3500.00 |    2000.00 |            0.07 |     442.00 |      3442.00 |         10 |
|          3 |         3676.06 |    3500.00 |    3000.00 |            0.07 |     234.06 |       176.06 |         10 |
|          4 |          188.03 |     188.03 |    7311.97 |            0.07 |      11.97 |         0.00 |         10 |
|          5 |            0.00 |       0.00 |   11811.97 |            0.07 |       0.00 |         0.00 |         10 |
|          6 |            0.00 |       0.00 |   16311.97 |            0.07 |       0.00 |         0.00 |         10 |
|          7 |            0.00 |       0.00 |   20811.97 |            0.07 |       0.00 |         0.00 |         10 |
|          8 |            0.00 |       0.00 |   25311.97

In [13]:
@printf("%119s", '\U2111')

                                                                                                                      ℑ